###### 2021-03-28

# Co-Routine (코루틴)
- 코루틴을 알아보기 전에 *메인함수*와 *서브함수*를 알아보자.

In [4]:
def sub_function(a, b):
    c = a + b
    print(c)
    print('서브 함수')

def main_function():
    sub_function(1, 2)
    print('메인 함수')

main_function()

3
서브 함수
메인 함수


![041001](https://user-images.githubusercontent.com/54063179/112755008-d5c56e00-9019-11eb-9ac1-df59eba60974.png)
- 위 그림처럼 메인함수와 서브함수의 관계에서는 `메인함수` > `서브함수` > `메인함수`로 코드를 실행한다.
- 또한, `서브함수`를 실행하고 다시 `메인함수`로 돌아오면, `서브함수`의 내용은 모두 사라지게 된다.

![041002](https://user-images.githubusercontent.com/54063179/112755090-276df880-901a-11eb-9887-cb49a87658df.png)
- 반면에 코루틴은 *Copoerative routine*의 줄임말로 서로 협력하는 루틴이다.
- 위에서 예시로 들었던 `메인함수`와 `서브함수`처럼 종속의 관계가 아니라 *서로 대등한 관계*이다.
- 그러므로 코루틴은 함수가 종료되지 않은 상태에서 메인 루틴의 코드를 실행한 뒤 다시 원래의 코드로 돌아온다. 이때, 코루틴이 종료되어도 코루틴의 내용이 사라지지 않는다.

## 1. 코루틴에 값 보내기
- 코루틴은 제너레이터의 한 종류이다.
- 제너레이터는 `yield`로 값을 발생시켰지만, 코루틴은 `yield`로 값을 받아올 수 있습니다.
- 다음과 같이 코루틴을 실행할때는 `send`메서드를 사용한다.
    - `코루틴객체.send(값)`
    - `변수 = (yield)`

In [7]:
def number_generator():
    while True:  # 코루틴을 무한히 사용하기위해 While True
        x = (yield)  # 코루틴 밖에서 send메서드를 이용해 값을 받아옴
        print(x)


co = number_generator()
next(co)  # 코루틴 안에 `yield` 까지 실행
          # 코루틴을 최초로 실행 하는 것이라고 보면됨
          # `co.send(None)`를 대신 사용해도 된다.
          # 자세한건 아래 그림을 참조하자

co.send(1)  # 코루틴에 숫자 1 보냄
co.send(2)  # 코루틴에 숫자 2 보냄
co.send(3)  # 코루틴에 숫자 3 보냄

1
2
3


- 아래는 위 코드의 실행 과정을 도식화한 것이다.
![041003](https://user-images.githubusercontent.com/54063179/112755406-808a5c00-901b-11eb-818a-55edb6972a2d.png)